In [ ]:
import cv2
import numpy as np
from sklearn.cluster import KMeans
import colorsys
import time

In [ ]:
def get_dominant_colors(image, k=5, image_processing_size=(100, 100)):
    """Extracts k dominant colors from an image using K-Means clustering."""
    if image is None:
        return []
    
    small_image = cv2.resize(image, image_processing_size, interpolation=cv2.INTER_AREA)
    pixels = small_image.reshape((-1, 3))
    pixels = np.float32(pixels)
    
    criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 10, 1.0)
    flags = cv2.KMEANS_RANDOM_CENTERS
    
    try:
        compactness, labels, centers = cv2.kmeans(pixels, k, None, criteria, 10, flags)
    except cv2.error as e:
        # Handle cases where k might be too large for the number of unique pixels
        # Fallback to a smaller k or return empty
        # For simplicity, we'll try with k=1 if the initial attempt fails
        try:
             compactness, labels, centers = cv2.kmeans(pixels, 1, None, criteria, 10, flags)
        except cv2.error:
             return [] # Return empty if even k=1 fails
        
    dominant_colors = np.uint8(centers)
    
    # Calculate percentage of each cluster
    counts = np.bincount(labels.flatten())
    percentages = counts / len(pixels)
    
    # Sort colors by percentage (descending)
    sorted_indices = np.argsort(percentages)[::-1]
    sorted_colors = dominant_colors[sorted_indices]
    sorted_percentages = percentages[sorted_indices]
    
    return list(zip(sorted_colors, sorted_percentages)) # Return BGR colors and their percentages

In [ ]:
def get_mood(colors_with_percentages):
    """Suggests a mood based on the dominant colors and their percentages."""
    if not colors_with_percentages:
        return "Neutral"

    total_hue = 0
    total_saturation = 0
    total_value = 0
    total_weight = 0

    for color_bgr, percentage in colors_with_percentages:
        # Convert BGR to HSV
        hsv = cv2.cvtColor(np.uint8([[color_bgr]]), cv2.COLOR_BGR2HSV)[0][0]
        hue, saturation, value = hsv[0], hsv[1] / 255.0, hsv[2] / 255.0
        
        # Weighted average (using percentage as weight)
        # Hue needs careful averaging (circular mean), but simple weighted average is often sufficient here
        total_hue += hue * percentage 
        total_saturation += saturation * percentage
        total_value += value * percentage
        total_weight += percentage

    if total_weight == 0: # Avoid division by zero
        return "Neutral"
        
    avg_hue = total_hue / total_weight
    avg_saturation = total_saturation / total_weight
    avg_value = total_value / total_weight

    # Mood mapping logic (example)
    if avg_saturation < 0.2:
        return "Muted / Calm"
    elif avg_value < 0.3:
        return "Dark / Mysterious"
    elif (0 <= avg_hue < 30) or (150 <= avg_hue <= 180): # Red/Orange/Pink range
        if avg_saturation > 0.6 and avg_value > 0.5:
            return "Energetic / Passionate"
        else:
            return "Warm / Cozy"
    elif 30 <= avg_hue < 75: # Yellow/Green range
         if avg_saturation > 0.5 and avg_value > 0.6:
            return "Cheerful / Natural"
         else:
            return "Earthy / Calm"
    elif 75 <= avg_hue < 150: # Cyan/Blue/Purple range
        if avg_saturation > 0.5 and avg_value > 0.4:
            return "Cool / Serene"
        else:
            return "Calm / Peaceful"
    else:
        return "Neutral / Balanced"

In [ ]:
def create_palette_image(colors_with_percentages, height=50):
    """Creates an image displaying color blocks based on percentages."""
    if not colors_with_percentages:
        return np.zeros((height, 300, 3), dtype=np.uint8)
        
    total_width = 300 # Fixed width for the palette bar
    palette = np.zeros((height, total_width, 3), dtype=np.uint8)
    current_x = 0
    for color, percentage in colors_with_percentages:
        block_width = int(percentage * total_width)
        if block_width <= 0 and percentage > 0: # Ensure even small percentages get at least 1 pixel
             block_width = 1
        
        end_x = current_x + block_width
        # Ensure we don't exceed total_width due to rounding
        if end_x > total_width:
            end_x = total_width
            block_width = total_width - current_x
            if block_width < 0:
                block_width = 0
        
        if block_width > 0:
             palette[:, current_x:end_x] = color # color is already BGR
        current_x = end_x
        
    # Fill any remaining space due to rounding errors with the last color
    if current_x < total_width and colors_with_percentages:
        last_color = colors_with_percentages[-1][0]
        palette[:, current_x:total_width] = last_color
        
    return palette

In [ ]:
def create_abstract_art(colors_with_percentages, height=200, width=300):
    """Generates simple abstract art (stripes/blocks) based on the color palette."""
    if not colors_with_percentages:
        return np.zeros((height, width, 3), dtype=np.uint8)

    art_image = np.zeros((height, width, 3), dtype=np.uint8)
    num_colors = len(colors_with_percentages)
    
    # Simple vertical stripes weighted by percentage
    current_x = 0
    for color, percentage in colors_with_percentages:
        stripe_width = int(percentage * width)
        if stripe_width <= 0 and percentage > 0:
            stripe_width = 1
            
        end_x = current_x + stripe_width
        if end_x > width:
            end_x = width
            stripe_width = width - current_x
            if stripe_width < 0:
                stripe_width = 0
        
        if stripe_width > 0:
            art_image[:, current_x:end_x] = color
        current_x = end_x
        
    # Fill remaining space
    if current_x < width and colors_with_percentages:
        last_color = colors_with_percentages[-1][0]
        art_image[:, current_x:width] = last_color

    return art_image

In [ ]:
# --- Configuration / Hyperparameters ---
NUM_CLUSTERS = 5 # K for K-Means (Hyperparameter)
WEBCAM_INDEX = 0
FRAME_WIDTH = 640
FRAME_HEIGHT = 480
ANALYSIS_INTERVAL = 0.5 # Seconds between analyses to reduce load
SHOW_ABSTRACT_ART = True # Optional feature flag

# --- Initialization ---
cap = cv2.VideoCapture(WEBCAM_INDEX)
cap.set(cv2.CAP_PROP_FRAME_WIDTH, FRAME_WIDTH)
cap.set(cv2.CAP_PROP_FRAME_HEIGHT, FRAME_HEIGHT)

if not cap.isOpened():
    print(f"Error: Could not open webcam index {WEBCAM_INDEX}.")
    # In a notebook, we might just stop here or raise an error
    # For this example, we'll let the loop handle the error display

last_analysis_time = time.time()
dominant_colors_cache = []
mood_cache = "Initializing..."
palette_img_cache = create_palette_image([])
art_img_cache = create_abstract_art([]) if SHOW_ABSTRACT_ART else None

# --- Main Loop ---
while True:
    ret, frame = cap.read()
    if not ret:
        print("Error: Failed to capture frame.")
        # Display a black screen or error message if capture fails
        frame = np.zeros((FRAME_HEIGHT, FRAME_WIDTH, 3), dtype=np.uint8)
        cv2.putText(frame, "Webcam Error", (50, FRAME_HEIGHT // 2), 
                    cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2, cv2.LINE_AA)
        # No analysis possible if frame capture fails
        dominant_colors_cache = []
        mood_cache = "Webcam Error"
        palette_img_cache = create_palette_image([])
        if SHOW_ABSTRACT_ART:
             art_img_cache = create_abstract_art([])
        # Still try to display something
        # break # Option to exit loop on error
    else:
        current_time = time.time()
        # --- Model Training (K-Means Fit) & Evaluation (Visual) ---
        if current_time - last_analysis_time >= ANALYSIS_INTERVAL:
            dominant_colors_cache = get_dominant_colors(frame, k=NUM_CLUSTERS)
            mood_cache = get_mood(dominant_colors_cache)
            palette_img_cache = create_palette_image(dominant_colors_cache)
            if SHOW_ABSTRACT_ART:
                art_img_cache = create_abstract_art(dominant_colors_cache)
            last_analysis_time = current_time

    # --- Results Visualization ---
    # Create combined output frame
    display_height = frame.shape[0]
    display_width = frame.shape[1]
    info_height = 100 # Height for palette and mood text
    art_height = 200 if SHOW_ABSTRACT_ART else 0
    art_width = 300 if SHOW_ABSTRACT_ART else 0
    
    # Ensure palette image matches expected width for display concatenation
    palette_display_width = display_width // 2 # Example: half width for palette
    resized_palette = cv2.resize(palette_img_cache, (palette_display_width, info_height // 2), interpolation=cv2.INTER_NEAREST)
    
    # Create info panel
    info_panel = np.zeros((info_height, display_width, 3), dtype=np.uint8)
    info_panel[0:resized_palette.shape[0], 0:resized_palette.shape[1]] = resized_palette
    cv2.putText(info_panel, f"Mood: {mood_cache}", (10, resized_palette.shape[0] + 30), 
                cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 255), 2, cv2.LINE_AA)

    # Combine frame and info panel
    combined_frame = cv2.vconcat([frame, info_panel])
    
    # Display main window
    cv2.imshow('ChromaMood - Live Feed & Analysis', combined_frame)

    # Display optional abstract art window
    if SHOW_ABSTRACT_ART and art_img_cache is not None:
        cv2.imshow('ChromaMood - Abstract Art', art_img_cache)

    # Exit condition
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# --- Cleanup ---
cap.release()
cv2.destroyAllWindows()

# In a Jupyter notebook, cv2.imshow might not work as expected.
# You might need to use alternative display methods like matplotlib or ipywidgets
# if running directly within the notebook environment without a separate window manager.
# However, the request asks for standard code, and cv2.imshow is standard for OpenCV apps.
# For notebook-native display, consider:
# from IPython.display import display, Image
# import io
# def display_cv_image(img, format='.jpeg'):
#     _, buf = cv2.imencode(format, img)
#     display(Image(data=buf.tobytes()))
# # Then call display_cv_image(combined_frame) inside the loop instead of cv2.imshow
# # This requires careful handling of the loop and updates within the notebook cell.
